# Ejercicio 1

In [1]:
import pandas as pd

class NaiveBayesClassifier:

    def train(self, df):
        grouped_classes = df.groupby('Nacionalidad')
        self.classes = df['Nacionalidad'].unique()
        class_amount = len(self.classes)
        self.class_probabilities = []
        self.attribute_probabilities = {}
        for _class, grouped in grouped_classes:
            data_len = len(grouped)
            total_len = len(df.index)
            self.class_probabilities.append(data_len / total_len)
            self.attribute_probabilities[_class] = []
            for attribute in df.columns[0:-1]:    # Skipping last column (Nacionalidad)
                self.attribute_probabilities[_class].append(grouped[attribute].mean())
            for i in range(len(self.attribute_probabilities[_class])):
                self.attribute_probabilities[_class][i] = (self.attribute_probabilities[_class][i] * total_len + 1) / (total_len + class_amount)
    
    def print_probabilities(self, values):
        user_probability = []
        j = 0
        for classname, probabilities in self.attribute_probabilities.items():
            user_probability.append(self.class_probabilities[j])
            for attribute_probability in probabilities:
                if values[j] == 1:
                    user_probability[j] *= attribute_probability
                else:
                    user_probability[j] *= (1 - attribute_probability)
            j += 1
        for value in zip(user_probability, self.classes):
            print(value)


file = 'inputs/PreferenciasBritanicos.xlsx'
brits_df = pd.read_excel(file)
classifier = NaiveBayesClassifier()
classifier.train(brits_df)
# 1.B
classifier.print_probabilities([1, 0, 1, 1, 1])
print()
# 1.C
classifier.print_probabilities([0, 1, 1, 0, 1])

(0.03716502587359729, 'I')
(0.018589743589743593, 'E')

(0.0015605655921302178, 'I')
(0.010256410256410256, 'E')
